In [1]:
#not working
import os
import cv2
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torch import nn, optim
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image


# Custom Dataset to load videos
import os
import cv2
import torch
import pandas as pd
from torch.utils.data import Dataset
import numpy as np

class VideoDataset(Dataset):
    def __init__(self, folder_path, labels_csv, transform=None):
        self.folder_path = folder_path
        self.labels = pd.read_csv(labels_csv)
        self.transform = transform
        
        # Convert video_summary to tuple of (x_summary, y_summary)
        self.labels['video_summary'] = self.labels['video_summary'].apply(eval) 

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        video_id = self.labels.iloc[idx]['video_id']
        video_path = os.path.join(self.folder_path, f"{video_id}.mp4")  # Ensure the filename matches `video_id`
        
        # OpenCV to read the video frames
        cap = cv2.VideoCapture(video_path)
        frames = []
        
        # Read 20 frames from the video
        for _ in range(20):
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
            frame = cv2.resize(frame, (64, 64))  # Resize to 64x64
            frames.append(frame)
        
        cap.release()
        
        frames = np.stack(frames)  # Stack to get shape (20, 64, 64, 3)
        
        # Convert frames to the proper format: (3, 20, 64, 64)
        frames = np.transpose(frames, (3, 0, 1, 2))  # Convert to (3, 20, 64, 64)
        
        frames = torch.tensor(frames, dtype=torch.float32)
        
        # Apply transformations if any
        if self.transform:
            frames = self.transform(frames)
        
        # Extract x_summary and y_summary
        x_summary, y_summary = self.labels.iloc[idx]['video_summary']
        
        return frames, torch.tensor([x_summary, y_summary], dtype=torch.float32)
# Define transformations (for tensor)
transform = transforms.Compose([
    transforms.Lambda(lambda x: x / 255.0)  # Normalize the frames to [0, 1]
])

# Load data and split 80% for training and 20% for testing
folder_path = 'BH25/Training_Data/Train_Videos'  # Change to your video folder path
labels_csv = 'BH25/Training_Data/train.csv'  # Path to your CSV file

dataset = VideoDataset(folder_path, labels_csv, transform=transform)
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# cnn to extract features
class Custom3DCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(Custom3DCNN, self).__init__()
        # Define a 3-layered 3D CNN
        self.conv1 = nn.Conv3d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm3d(32)
        self.relu = nn.ReLU()
        
        self.conv2 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm3d(64)
        
        self.conv3 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm3d(128)
        
        self.pool = nn.AdaptiveAvgPool3d(1)  # Global average pooling
        
        # Fully connected layer
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        # Forward pass through the 3D CNN
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        
        # Global average pooling
        x = self.pool(x)
        x = torch.flatten(x, 1)  # Flatten for the fully connected layer
        x = self.fc(x)
        return x

# Check if CUDA (GPU) is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model and move it to the appropriate device
model = Custom3DCNN(num_classes=2).to(device)

# Loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for frames, labels in train_loader:
        # Move frames and labels to the same device as the model (GPU or CPU)
        frames, labels = frames.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(frames)
        
        # Compute loss
        loss = loss_fn(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
        # Evaluate on validation data
    # model.eval()  # Evaluation mode
    # val_loss = 0.0
    # with torch.no_grad():
    #     for frames, labels in test_loader:
    #         frames, labels = frames.to(device), labels.to(device)
    #         outputs = model(frames)
    #         val_loss += loss_fn(outputs, labels).item()

    avg_val_loss = val_loss / len(test_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'resnet3d_model2.pth')


Epoch 1/10, Loss: 238.7252, Val Loss: 239.2298
Epoch 2/10, Loss: 238.5590, Val Loss: 239.4200


KeyboardInterrupt: 

In [ ]:
import torch
from sklearn.metrics import mean_absolute_error
import numpy as np

# Check if CUDA is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the trained model
model = Custom3DCNN(num_classes=2)
model.load_state_dict(torch.load('resnet3d_model2.pth'))
model.to(device)  # Move the model to the selected device
model.eval()

# Initialize lists to store predictions and true labels
all_preds = []
all_labels = []

# Evaluate the model
with torch.no_grad():
    for frames, labels in test_loader:
        frames, labels = frames.to(device), labels.to(device)  # Move data to the selected device
        
        # Forward pass to get predictions
        outputs = model(frames)
        
        # Collect predictions and true labels
        all_preds.append(outputs.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Convert lists to numpy arrays for evaluation
all_preds = np.concatenate(all_preds, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

# Calculate Mean Absolute Error (MAE)
mae_loss = mean_absolute_error(all_labels, all_preds)
print(f"Mean Absolute Error (MAE): {mae_loss:.4f}")